<a href="https://colab.research.google.com/github/syphax/running-analysis/blob/main/XC_Race_Result_Compilations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install requests

In [ ]:
import requests
import pandas as pd
import numpy as np

In [ ]:
def get_milesplit_results(url):
  '''
  url: Complete URL for results, complete with parameters

  Yes, I know I should break the url into the endpoint and parameters,
  but for v0.1 I'm just using monolithic URLs
  '''

  response = requests.get(url)

  # Check if the request was successful
  if response.status_code == 200:
      # Parse the response JSON if the content type is correct
      if 'application/json' in response.headers.get('Content-Type'):

          # Get the full response
          full_response = response.json()

          # Get the data grid and make a DataFrame
          df_result_full = pd.DataFrame(full_response['data'])

          # Drop some fields
          df_result = df_result_full.drop(['teamProfileUrl', 'profileUrl', 'teamLogo'], axis=1)


          print('Success getting data from {}'.format(url))
      else:
          print('Response content is not in JSON format.')
          df_result = None
  else:
      print(f'Error: {response.status_code}')
      print(response.text)
      df_result = None

  return df_result

In [ ]:
# Thanks ChatGPT:

def convert_to_timedelta(time_str):
    # Split the string into minutes, seconds, and hundredths
    minutes, seconds = time_str.split(':')
    seconds, hundredths = seconds.split('.')

    # Convert each part to timedelta
    minutes_delta = pd.Timedelta(minutes=int(minutes))
    seconds_delta = pd.Timedelta(seconds=int(seconds))
    hundredths_delta = pd.Timedelta(milliseconds=int(hundredths) * 10)

    # Sum up the deltas to get the total timedelta
    total_delta = minutes_delta + seconds_delta + hundredths_delta
    return total_delta

# My go-to column name flattener:

def flatten_columns(df, joiner='_'):
    df.columns = df.columns.map(joiner.join).str.strip(joiner)
    return df

In [ ]:
# List of dicts of races:

race_dict = {'Cape Ann':'https://ma.milesplit.com/api/v1/meets/578420/performances?resultsId=986788&fields=id%2CmeetId%2CmeetName%2CteamId%2CvideoId%2CteamName%2CathleteId%2CfirstName%2ClastName%2Cgender%2CgenderName%2CdivisionId%2CdivisionName%2CageGroupName%2CgradYear%2CeventName%2CeventCode%2CeventDistance%2CeventGenreOrder%2Cround%2CroundName%2Cheat%2Cunits%2Cmark%2Cplace%2CwindReading%2CprofileUrl%2CteamProfileUrl%2C+performanceVideoId%2C+teamLogo&teamScores=true',
             'Bay State':'https://ma.milesplit.com/api/v1/meets/577861/performances?resultsId=986854&fields=id%2CmeetId%2CmeetName%2CteamId%2CvideoId%2CteamName%2CathleteId%2CfirstName%2ClastName%2Cgender%2CgenderName%2CdivisionId%2CdivisionName%2CageGroupName%2CgradYear%2CeventName%2CeventCode%2CeventDistance%2CeventGenreOrder%2Cround%2CroundName%2Cheat%2Cunits%2Cmark%2Cplace%2CwindReading%2CprofileUrl%2CteamProfileUrl%2C+performanceVideoId%2C+teamLogo&teamScores=true',
             'Middlesex':'https://ma.milesplit.com/api/v1/meets/580397/performances?resultsId=987179&fields=id%2CmeetId%2CmeetName%2CteamId%2CvideoId%2CteamName%2CathleteId%2CfirstName%2ClastName%2Cgender%2CgenderName%2CdivisionId%2CdivisionName%2CageGroupName%2CgradYear%2CeventName%2CeventCode%2CeventDistance%2CeventGenreOrder%2Cround%2CroundName%2Cheat%2Cunits%2Cmark%2Cplace%2CwindReading%2CprofileUrl%2CteamProfileUrl%2C+performanceVideoId%2C+teamLogo&teamScores=true',
             'Cape and Islands':'https://ma.milesplit.com/api/v1/meets/579796/performances?resultsId=986665&fields=id%2CmeetId%2CmeetName%2CteamId%2CvideoId%2CteamName%2CathleteId%2CfirstName%2ClastName%2Cgender%2CgenderName%2CdivisionId%2CdivisionName%2CageGroupName%2CgradYear%2CeventName%2CeventCode%2CeventDistance%2CeventGenreOrder%2Cround%2CroundName%2Cheat%2Cunits%2Cmark%2Cplace%2CwindReading%2CprofileUrl%2CteamProfileUrl%2C+performanceVideoId%2C+teamLogo&teamScores=true',
             'Catholic Mem Invite':'https://ma.milesplit.com/api/v1/meets/578923/performances?resultsId=979826&fields=id%2CmeetId%2CmeetName%2CteamId%2CvideoId%2CteamName%2CathleteId%2CfirstName%2ClastName%2Cgender%2CgenderName%2CdivisionId%2CdivisionName%2CageGroupName%2CgradYear%2CeventName%2CeventCode%2CeventDistance%2CeventGenreOrder%2Cround%2CroundName%2Cheat%2Cunits%2Cmark%2Cplace%2CwindReading%2CprofileUrl%2CteamProfileUrl%2C+performanceVideoId%2C+teamLogo&teamScores=true',
             'Patriot':'https://ma.milesplit.com/api/v1/meets/572358/performances?resultsId=986745&fields=id%2CmeetId%2CmeetName%2CteamId%2CvideoId%2CteamName%2CathleteId%2CfirstName%2ClastName%2Cgender%2CgenderName%2CdivisionId%2CdivisionName%2CageGroupName%2CgradYear%2CeventName%2CeventCode%2CeventDistance%2CeventGenreOrder%2Cround%2CroundName%2Cheat%2Cunits%2Cmark%2Cplace%2CwindReading%2CprofileUrl%2CteamProfileUrl%2C+performanceVideoId%2C+teamLogo&teamScores=true',
             'Merrimack':'https://ma.milesplit.com/api/v1/meets/580308/performances?resultsId=986653&fields=id%2CmeetId%2CmeetName%2CteamId%2CvideoId%2CteamName%2CathleteId%2CfirstName%2ClastName%2Cgender%2CgenderName%2CdivisionId%2CdivisionName%2CageGroupName%2CgradYear%2CeventName%2CeventCode%2CeventDistance%2CeventGenreOrder%2Cround%2CroundName%2Cheat%2Cunits%2Cmark%2Cplace%2CwindReading%2CprofileUrl%2CteamProfileUrl%2C+performanceVideoId%2C+teamLogo&teamScores=true',
             'Dual County Thorpe 2':' https://ma.milesplit.com/api/v1/meets/572592/performances?resultsId=985976&fields=id%2CmeetId%2CmeetName%2CteamId%2CvideoId%2CteamName%2CathleteId%2CfirstName%2ClastName%2Cgender%2CgenderName%2CdivisionId%2CdivisionName%2CageGroupName%2CgradYear%2CeventName%2CeventCode%2CeventDistance%2CeventGenreOrder%2Cround%2CroundName%2Cheat%2Cunits%2Cmark%2Cplace%2CwindReading%2CprofileUrl%2CteamProfileUrl%2C+performanceVideoId%2C+teamLogo&teamScores=true',
             'Dual County Thorpe 1':'https://ma.milesplit.com/api/v1/meets/572592/performances?resultsId=985975&fields=id%2CmeetId%2CmeetName%2CteamId%2CvideoId%2CteamName%2CathleteId%2CfirstName%2ClastName%2Cgender%2CgenderName%2CdivisionId%2CdivisionName%2CageGroupName%2CgradYear%2CeventName%2CeventCode%2CeventDistance%2CeventGenreOrder%2Cround%2CroundName%2Cheat%2Cunits%2Cmark%2Cplace%2CwindReading%2CprofileUrl%2CteamProfileUrl%2C+performanceVideoId%2C+teamLogo&teamScores=true',
             'Dual County Foley':'https://ma.milesplit.com/api/v1/meets/572592/performances?resultsId=985973&fields=id%2CmeetId%2CmeetName%2CteamId%2CvideoId%2CteamName%2CathleteId%2CfirstName%2ClastName%2Cgender%2CgenderName%2CdivisionId%2CdivisionName%2CageGroupName%2CgradYear%2CeventName%2CeventCode%2CeventDistance%2CeventGenreOrder%2Cround%2CroundName%2Cheat%2Cunits%2Cmark%2Cplace%2CwindReading%2CprofileUrl%2CteamProfileUrl%2C+performanceVideoId%2C+teamLogo&teamScores=true',
             'Hockomock':'https://ma.milesplit.com/api/v1/meets/572508/performances?resultsId=986801&fields=id%2CmeetId%2CmeetName%2CteamId%2CvideoId%2CteamName%2CathleteId%2CfirstName%2ClastName%2Cgender%2CgenderName%2CdivisionId%2CdivisionName%2CageGroupName%2CgradYear%2CeventName%2CeventCode%2CeventDistance%2CeventGenreOrder%2Cround%2CroundName%2Cheat%2Cunits%2Cmark%2Cplace%2CwindReading%2CprofileUrl%2CteamProfileUrl%2C+performanceVideoId%2C+teamLogo&teamScores=true',

             }

locations_dict = {'Cape Ann':'Wrentham',
                  'Bay State':'Wrentham',
                  'Middlesex':'Woburn Country Club',
                  'Cape and Islands':'Dennis-Yarmouth',
                  'Catholic Mem Invite':'Franklin Park',
                  'Patriot':'Hingham',
                  'Merrimack':'Shedd Park',
                  'Dual County Thorpe 2':'Franklin Park',
                  'Dual County Thorpe 1':'Franklin Park',
                  'Dual County Foley':'Franklin Park',
                  'Hockomock':'Wrentham'
                  }


In [ ]:
list_results = []

for r in race_dict.keys():
  print(r)
  result = get_milesplit_results(race_dict[r])
  result['Race Key'] = r
  result['Location'] = locations_dict[r]

  list_results.append(result)

df_results = pd.concat(list_results, axis=0)


Cape Ann
Success getting data from https://ma.milesplit.com/api/v1/meets/578420/performances?resultsId=986788&fields=id%2CmeetId%2CmeetName%2CteamId%2CvideoId%2CteamName%2CathleteId%2CfirstName%2ClastName%2Cgender%2CgenderName%2CdivisionId%2CdivisionName%2CageGroupName%2CgradYear%2CeventName%2CeventCode%2CeventDistance%2CeventGenreOrder%2Cround%2CroundName%2Cheat%2Cunits%2Cmark%2Cplace%2CwindReading%2CprofileUrl%2CteamProfileUrl%2C+performanceVideoId%2C+teamLogo&teamScores=true
Bay State
Success getting data from https://ma.milesplit.com/api/v1/meets/577861/performances?resultsId=986854&fields=id%2CmeetId%2CmeetName%2CteamId%2CvideoId%2CteamName%2CathleteId%2CfirstName%2ClastName%2Cgender%2CgenderName%2CdivisionId%2CdivisionName%2CageGroupName%2CgradYear%2CeventName%2CeventCode%2CeventDistance%2CeventGenreOrder%2Cround%2CroundName%2Cheat%2Cunits%2Cmark%2Cplace%2CwindReading%2CprofileUrl%2CteamProfileUrl%2C+performanceVideoId%2C+teamLogo&teamScores=true
Middlesex
Success getting data fr

In [ ]:
# Additional Info

df_results['Full Name'] = df_results['firstName'] + ' ' + df_results['lastName']

In [ ]:
# Fixes

df_results['Race Name'] = np.where(df_results['Race Key'].isin(['Dual County Thorpe 1', 'Dual County Thorpe 2', 'Dual County Foley']), 'Dual County', df_results['Race Key'])

In [ ]:
# Convert marks to time deltas:

df_results['Time'] = df_results['mark'].apply(convert_to_timedelta)

df_results['Time_str'] = df_results['Time'].astype(str).map(lambda x: x[7:18])
#df_results['Time'] = df_results['mark'].apply(lambda x: pd.to_timedelta(x))


In [ ]:
df_results[['mark', 'Time', 'Time_str']].sample(5)

,mark,Time,Time_str
698,21:34.10,0 days 00:21:34.100000,00:21:34.10
671,20:52.40,0 days 00:20:52.400000,00:20:52.40
690,21:27.20,0 days 00:21:27.200000,00:21:27.20
256,11:08.30,0 days 00:11:08.300000,00:11:08.30
215,18:38.60,0 days 00:18:38.600000,00:18:38.60


# Trim and Clean Data

In [ ]:
df_results_bak = df_results.copy()

In [ ]:
print(df_results.shape)
df_results = df_results[df_results['gender']=='M']
print(df_results.shape)
df_results = df_results[df_results['eventCode']=='5000m']
print(df_results.shape)

(2283, 33)
(1599, 33)
(1380, 33)


In [ ]:
df_results = df_results.sort_values(['Race Name', 'mark'], ascending=True)

In [ ]:
df_results['Team Place'] = df_results.groupby(['Race Name', 'teamName']).cumcount() + 1

In [ ]:
df_results_top7 = df_results[df_results['Team Place'] <= 7]

In [ ]:
df_results_team = df_results_top7.groupby(['Race Name', 'Location', 'teamName'], as_index=False).agg(
    {'athleteId':'nunique', 'Time':['min','mean','max']})

df_results_team = flatten_columns(df_results_team)

df_results_team['Runners'] = df_results_team['athleteId_nunique']
df_results_team['Time-fastest'] = df_results_team['Time_min'].astype(str).map(lambda x: x[7:18])
df_results_team['Time-mean'] = df_results_team['Time_mean'].astype(str).map(lambda x: x[7:18])
df_results_team['Time-slowest'] = df_results_team['Time_max'].astype(str).map(lambda x: x[7:18])

df_results_team_formatted = df_results_team[['teamName', 'Race Name', 'Location', 'Runners', 'Time-fastest', 'Time-mean', 'Time-slowest']]


In [ ]:
df_results_team

,Race Name,Location,teamName,athleteId_nunique,Time_min,Time_mean,Time_max,Runners,Time-fastest,Time-mean,Time-slowest
0,Bay State,Wrentham,Braintree High School,7,0 days 00:17:44.200000,0 days 00:18:49.600000,0 days 00:19:44.300000,7,00:17:44.20,00:18:49.60,00:19:44.30
1,Bay State,Wrentham,Brookline High School,7,0 days 00:16:00.600000,0 days 00:16:28.128571428,0 days 00:17:13,7,00:16:00.60,00:16:28.12,00:17:13
2,Bay State,Wrentham,Framingham High School,7,0 days 00:17:49.300000,0 days 00:18:48.800000,0 days 00:20:03.300000,7,00:17:49.30,00:18:48.80,00:20:03.30
3,Bay State,Wrentham,Milton High School,7,0 days 00:17:34,0 days 00:19:03.357142857,0 days 00:20:40.100000,7,00:17:34,00:19:03.35,00:20:40.10
4,Bay State,Wrentham,Natick High School,7,0 days 00:16:23.600000,0 days 00:16:56.814285714,0 days 00:17:43,7,00:16:23.60,00:16:56.81,00:17:43
...,...,...,...,...,...,...,...,...,...,...,...
125,Patriot,Hingham,Plymouth North High School,7,0 days 00:17:07.910000,0 days 00:19:12.740000,0 days 00:22:27.740000,7,00:17:07.91,00:19:12.74,00:22:27.74
126,Patriot,Hingham,Plymouth South High School,7,0 days 00:16:02.910000,0 days 00:17:15.424285714,0 days 00:18:58.610000,7,00:16:02.91,00:17:15.42,00:18:58.61
127,Patriot,Hingham,Scituate High School,7,0 days 00:17:23.890000,0 days 00:19:10.431428571,0 days 00:21:01.590000,7,00:17:23.89,00:19:10.43,00:21:01.59
128,Patriot,Hingham,Silver Lake Regional High School,7,0 days 00:17:48.790000,0 days 00:18:45.535714285,0 days 00:19:36.160000,7,00:17:48.79,00:18:45.53,00:19:36.16


In [ ]:
df_results_top7[df_results_top7['teamName']==' High School']

,id,eventCode,gender,athleteId,meetId,teamId,round,heat,place,windReading,...,eventGenreOrder,performanceVideoId,mark,Race Key,Location,Full Name,Race Name,Time,Time_str,Team Place


In [ ]:
df_results_team_formatted.to_csv('team_results_formatted.csv')

In [ ]:
df_results_top7.to_csv('individual_results.csv')

# Checks and Diagnostics

In [ ]:
df_results.groupby('Race Name').agg({'id':'count'})

,id
Race Name,
Bay State,173
Cape Ann,70
Cape and Islands,108
Catholic Mem Invite,426
Dual County,75
Hockomock,107
Merrimack,81
Middlesex,246
Patriot,94


In [ ]:
df_results.groupby(['Race Name', 'meetId']).agg({'id':'count', 'Full Name':'nunique'})

,,id,Full Name
Race Name,meetId,,
Bay State,577861,173,173
Cape Ann,578420,70,70
Cape and Islands,579796,108,108
Catholic Mem Invite,578923,426,426
Dual County,572592,75,75
Hockomock,572508,107,107
Merrimack,580308,81,81
Middlesex,580397,246,246
Patriot,572358,94,93


In [ ]:
df_results[df_results['meetId']=='578923'].groupby(['eventCode', 'gender', 'round', 'heat', 'divisionId']).agg({'id':'count', 'Full Name':'nunique'})


id  Full Name
eventCode gender round heat divisionId                
5000m     M      f     1    110         140        140
                            72          178        178
                            92          108        108

In [ ]:
df_results[df_results['meetId']=='578923'].sample(10).T


,528,527,722,603,440,680,798,545,393,754
id,151731696,151732072,151731653,151732103,151731531,151732135,151732226,151732079,151731452,151732188
eventCode,5000m,5000m,5000m,5000m,5000m,5000m,5000m,5000m,5000m,5000m
gender,M,M,M,M,M,M,M,M,M,M
athleteId,11339945,11432223,9279109,12545126,11178982,13985178,13956634,12659434,10248410,13937906
meetId,578923,578923,578923,578923,578923,578923,578923,578923,578923,578923
teamId,19674,15203,19429,19179,14375,19366,19198,19297,14710,19316
round,f,f,f,f,f,f,f,f,f,f
heat,1,1,1,1,1,1,1,1,1,1
place,39,13,136,44,43,76,167,20,6,129
windReading,,,,,,,,,,


In [ ]:
df_results.head(2).T

,142,143
id,152238976,152238977
eventCode,5000m,5000m
gender,M,M
athleteId,13978101,12408079
meetId,577861,577861
teamId,19337,19417
round,f,f
heat,1,1
place,1,2
windReading,,


In [ ]:
df_results[df_results['lastName']=='Crounse'].T


,21
id,152160294
eventCode,5000m
gender,M
athleteId,12575365
meetId,572592
teamId,19231
round,f
heat,1
place,29
windReading,


In [ ]:
df_results.dtypes

id                             object
eventCode                      object
gender                         object
athleteId                      object
meetId                         object
teamId                         object
round                          object
heat                           object
place                          object
windReading                    object
units                          object
teamName                       object
firstName                      object
lastName                       object
gradYear                       object
eventName                      object
eventDistance                  object
ageGroupName                   object
meetName                       object
videoId                         int64
divisionId                     object
divisionName                   object
roundName                      object
genderName                     object
eventGenreOrder                 int64
performanceVideoId             object
mark        

# Reference

In [ ]:
# FOR REFERENCE


# # The URL of the API endpoint
# url = 'https://ma.milesplit.com/api/v1/meets/578420/performances'

# url = 'https://ma.milesplit.com/api/v1/meets/578420/performances?resultsId=986788&fields=id%2CmeetId%2CmeetName%2CteamId%2CvideoId%2CteamName%2CathleteId%2CfirstName%2ClastName%2Cgender%2CgenderName%2CdivisionId%2CdivisionName%2CageGroupName%2CgradYear%2CeventName%2CeventCode%2CeventDistance%2CeventGenreOrder%2Cround%2CroundName%2Cheat%2Cunits%2Cmark%2Cplace%2CwindReading%2CprofileUrl%2CteamProfileUrl%2C+performanceVideoId%2C+teamLogo&teamScores=true'


# # Any query parameters or headers you need to send with the request
# params = {'resultsID': '986788',
#           'fields': 'id%2CmeetId%2CmeetName%2CteamId%2CvideoId%2CteamName%2CathleteId%2CfirstName%2ClastName%2Cgender%2CgenderName%2CdivisionId%2CdivisionName%2CageGroupName%2CgradYear%2CeventName%2CeventCode%2CeventDistance%2CeventGenreOrder%2Cround%2CroundName%2Cheat%2Cunits%2Cmark%2Cplace%2CwindReading%2CprofileUrl%2CteamProfileUrl%2C+performanceVideoId%2C+teamLogo',
#           'teamScores': 'true'}

# #headers = {'Authorization': 'Bearer YOUR_ACCESS_TOKEN'}

# # Make the GET request
# response = requests.get(url) # , params=params) # , headers=headers)

# # Check if the request was successful
# if response.status_code == 200:
#     # Parse the response JSON if the content type is correct
#     if 'application/json' in response.headers.get('Content-Type'):
#         data = response.json()
#         print(data)
#     else:
#         print('Response content is not in JSON format.')
# else:
#     print(f'Error: {response.status_code}')
#     print(response.text)
